In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from src.data_loader import get_data
from src.model import DyanFHEGPIAN
from src.train import create_sequences, SEQ_LEN, TARGET_COL, DEVICE
from src.utils import plot_predictions, calculate_metrics, create_results_dir

def predict_future(model, data_scaled, num_days, seq_len, scaler_target):
    """
    Generates future predictions by iteratively feeding the last prediction back into the model.
    """
    # Get the last known sequence from the data
    last_sequence = data_scaled[-seq_len:]
    future_predictions = []

    current_sequence = torch.tensor(last_sequence, dtype=torch.float32).unsqueeze(0).to(DEVICE)

    print(f"Generating predictions for the next {num_days} days...")
    for _ in range(num_days):
        model.eval()
        with torch.no_grad():
            # Get the prediction (scaled value)
            next_pred_scaled = model(current_sequence)[0]

            # Inverse transform to see the actual value (for inspection)
            # next_pred_actual = scaler_target.inverse_transform(next_pred_scaled.cpu().numpy())
            # print(f"Predicted Day {_+1}: {next_pred_actual[0][0]:.2f}")

            # Append the scaled prediction to our list
            future_predictions.append(next_pred_scaled.cpu().numpy()[0, 0])

            # Create the new input for the next prediction
            # The new input is the last `seq_len-1` observations plus the new prediction
            # We need to create a full feature vector for the new time step.
            # This is a simplification: we'll use the predicted 'Close' and carry over other features.
            new_row = current_sequence.cpu().numpy()[0, -1, :].copy()
            new_row[-1] = next_pred_scaled.cpu().numpy()[0, 0] # Update the target column

            new_sequence_np = np.vstack([current_sequence.cpu().numpy()[0, 1:, :], new_row])
            current_sequence = torch.tensor(new_sequence_np, dtype=torch.float32).unsqueeze(0).to(DEVICE)

    return np.array(future_predictions)

def main():
    """Main function to load model and make predictions."""
    create_results_dir()

    # --- 1. Load and Prepare Data ---
    df = get_data(ticker="MSFT")

    # We need the full dataset to create the last sequence for future prediction
    features = df.drop(columns=[TARGET_COL])
    target = df[[TARGET_COL]]

    scaler_features = MinMaxScaler()
    scaler_target = MinMaxScaler()

    features_scaled = scaler_features.fit_transform(features)
    target_scaled = scaler_target.fit_transform(target)

    data_scaled = np.concatenate([features_scaled, target_scaled], axis=1)

    # --- 2. Load the Trained Model ---
    # We need to know the best hyperparameters to instantiate the model architecture
    # For this script, we'll use some default good parameters.
    # In a real pipeline, you would save/load these from the training run.
    best_params = {
        'lr': 0.001, 'embed_dim': 64, 'hidden_dim': 128,
        'num_heads': 4, 'lambda_physics': 0.1
    }

    input_dim = data_scaled.shape[1]
    maxvit_params = {'embed_dim': best_params['embed_dim'], 'num_heads': best_params['num_heads'], 'num_blocks': 2}
    feinn_params = {'hidden_dim': best_params['hidden_dim'], 'output_dim': 1, 'n_layers': 2}
    dhgann_params = {'hidden_dim': best_params['hidden_dim'], 'output_dim': best_params['hidden_dim'] // 2, 'num_heads': best_params['num_heads']}

    model = DyanFHEGPIAN(
        input_dim=input_dim,
        seq_len=SEQ_LEN,
        maxvit_params=maxvit_params,
        feinn_params=feinn_params,
        dhgann_params=dhgann_params
    ).to(DEVICE)

    try:
        model.load_state_dict(torch.load('dyan_fheg_pian_model.pth', map_location=DEVICE))
        print("Trained model loaded successfully.")
    except FileNotFoundError:
        print("Error: Trained model 'dyan_fheg_pian_model.pth' not found.")
        print("Please run 'python src/train.py' first.")
        return

    model.eval()

    # --- 3. Predict on Test Set & Future ---
    # Predict on the test set (last 20% of data)
    _, X_test = create_sequences(data_scaled, SEQ_LEN)
    test_len = int(len(X_test) * 0.2)
    X_test_subset = torch.tensor(X_test[-test_len:], dtype=torch.float32).to(DEVICE)

    with torch.no_grad():
        test_preds_scaled = model(X_test_subset)[0]

    test_preds = scaler_target.inverse_transform(test_preds_scaled.cpu().numpy())
    actuals = df[TARGET_COL].values[-len(test_preds):]

    # Generate future predictions
    # Predict for roughly one year (252 trading days)
    future_preds_scaled = predict_future(model, data_scaled, num_days=252, seq_len=SEQ_LEN, scaler_target=scaler_target)
    future_preds = scaler_target.inverse_transform(future_preds_scaled.reshape(-1, 1))

    # --- 4. Visualize Results ---
    # Combine actuals, test predictions, and future predictions for plotting

    # Dates for plotting
    test_dates = df.index[-len(test_preds):]
    last_date = df.index[-1]
    future_dates = pd.to_datetime(pd.date_range(start=last_date + pd.Timedelta(days=1), periods=len(future_preds)))

    # Plotting
    plt.figure(figsize=(18, 8))
    plt.plot(df.index, df[TARGET_COL], label='Historical Actual Price', color='black')
    plt.plot(test_dates, test_preds, label='Model Forecast on Test Data', color='orange', linestyle='--')
    plt.plot(future_dates, future_preds, label='Future Forecast (2025)', color='red', linestyle='--')

    plt.title(f'MSFT Stock Price Forecast using Dyan-FHEG-PIAN')
    plt.xlabel('Date')
    plt.ylabel('Stock Price (USD)')
    plt.legend()
    plt.grid(True)
    plt.savefig('results/full_forecast.png')
    plt.show()

    print("\n--- Test Set Performance ---")
    calculate_metrics(actuals, test_preds)
    print("Prediction plot saved to results/full_forecast.png")

if __name__ == '__main__':
    main()